In [1]:
!wget https://raw.githubusercontent.com/congdaoduy298/Crawl-Data/master/data.json
!wget https://raw.githubusercontent.com/congdaoduy298/Crawl-Data/master/VN_name_entity.json

--2020-09-12 13:29:04--  https://raw.githubusercontent.com/congdaoduy298/Crawl-Data/master/data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5111581 (4.9M) [text/plain]
Saving to: ‘data.json’

data.json           100%[===================>]   4.87M  --.-KB/s    in 0.1s    

2020-09-12 13:29:04 (48.8 MB/s) - ‘data.json’ saved [5111581/5111581]

--2020-09-12 13:29:04--  https://raw.githubusercontent.com/congdaoduy298/Crawl-Data/master/VN_name_entity.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16971 (17K) [text/plain]
Saving to

In [2]:
!pip install transformers

     |████████████████████████████████| 890kB 2.9MB/s 
     |████████████████████████████████| 3.0MB 9.6MB/s 
     |████████████████████████████████| 1.1MB 30.6MB/s 
     |████████████████████████████████| 890kB 40.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=62667dddbb96f39262af771f1e444151198cb55b0e175fd78b5ac0cf160b0740
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
     "O",       # Outside of a named entity
     "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
     "I-MISC",  # Miscellaneous entity
     "B-PER",   # Beginning of a person's name right after another person's name
     "I-PER",   # Person's name
     "B-ORG",   # Beginning of an organisation right after another organisation
     "I-ORG",   # Organisation
     "B-LOC",   # Beginning of a location right after another location
     "I-LOC"    # Location
 ]

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very" \
            "close to the Manhattan Bridge."

In [4]:
def most_frequent(List): 
    return max(set(List), key = List.count) 

def join_token(predictions, tokens):
  # Join token to word 
  words = []
  i = 0
  n = len(predictions)
  while i<n:
    j = i
    entity = tokens[j]
    while j<n-1 and '##' in tokens[j+1]:
      token = tokens[j+1].split('##')[-1]
      entity += token
      j += 1
    if len(words)>0 and words[-1][0] in predictions[i:j+1]:
      words.append([words[-1][0], entity, i, j])
    else:
      words.append([most_frequent(predictions[i:j+1].tolist()), entity, i, j])
    i = j+1  
  return words

def join_word(words):
  # Join word to named entity
  entity = ''
  entities = []
  for i in range(len(words)):
    if words[i][0] != 0:
      if (len(entities)>0) and entities[-1][0] == words[i][0] \
          and (words[i][2] == entities[-1][3]+1 \
          or (words[i][2] == entities[-1][3]+2 \
          and tokens[words[i][2]-1] in ['.','-'])):
        entities[-1][1] += f' {words[i][1]}'
        entities[-1][3] = words[i][3]
      else:
        if words[i][1] != '[CLS]':
          entities.append(words[i])
  return entities

In [5]:
from tqdm.notebook import tqdm
import json
import time 
import json
import sys 
import re 

def add_to_dict(entity, ner_dict):
    entity = re.sub(r'\_', ' ', entity)
    if entity not in ner_dict:
        ner_dict[entity] = 1
    else:
        ner_dict[entity] += 1

def get_dict_ner(result, ner_dict):
    ner = []
    n_sentences = len(result)

    for i in range(n_sentences):
        for i, dict_word in enumerate(result[i]):
            if dict_word['nerLabel'] != 'O':
                try:
                    if dict_word['nerLabel'].split('-')[1] == 'MISC':
                        continue
                    if len(ner) > 0 and (ner[-1][1] == i-1) and next_cate[ner[-1][2]] == dict_word['nerLabel']:
                        ner[-1][0] += ' ' + dict_word['form']
                        ner[-1][1] += 1
                    else:
                        ner.append([dict_word['form'], i, dict_word['nerLabel']])
                except Exception as e:
                    print(e)

    for name_entity in ner:
        add_to_dict(name_entity[0], ner_dict)

def is_vietnamese(text, vowels):
    if "This review has been hidden" in text:
        text = text.split("This review has been hidden because it contains spoilers. To view it, click here.")[1]
    try:
        five_word = text.split(' ')[:20]
    except:
        five_word = text.split(' ')
    for word in five_word:
        for ch in word:
            if ch.lower() in vowels:
                return True 
    return False 
                

def normalize(txt):
    txt = re.sub(r'[\.\n]', '. ', txt)
    txt = re.sub(r'[\-]', ' ', txt)
    txt = re.sub(r'[\(\)]', ' ', txt)   
    txt = re.sub(r'\s+', ' ', txt)
    return txt 

In [8]:
if __name__ == "__main__":

    # 1. Connect to VnCoreNLP server
    # annotator = VnCoreNLP(address="http://127.0.0.1", port=9000) 

    with open('/content/data.json', 'r') as f:
        data = json.load(f)

    with open('/content/VN_name_entity.json', 'r') as f:
        ner_dict = json.load(f)

    count = 0
    vowels = ['á', 'ạ', 'à', 'ả', 'ã', 
              'ă', 'ắ', 'ặ', 'ằ', 'ẳ', 'ẵ',
              'â', 'ấ', 'ậ', 'ầ', 'ẩ', 'ẫ',
              'đ', 'é', 'ẹ', 'è', 'ẻ', 'ẽ',
              'ê', 'ế', 'ệ', 'ề', 'ể', 'ễ',
              'í', 'ị', 'ì', 'ỉ', 'ĩ',
              'ó', 'ọ', 'ò', 'ỏ', 'õ',
              'ô', 'ố', 'ộ', 'ồ', 'ổ', 'ỗ',
              'ơ', 'ớ', 'ợ', 'ờ', 'ở', 'ỡ',
              'ú', 'ụ', 'ù', 'ủ', 'ũ',
              'ý', 'ỵ', 'ỳ', 'ỷ', 'ỹ']
    CHOOSE_LIST = ['NORP', 'ORG', 'PERSON', 'LOC', 'GPE']
    next_cate = {'B-PER': 'I-PER', 'B-ORG': 'I-ORG', 'B-LOC': 'I-LOC', 'I-PER': 0, 'I-ORG': 0, 'I-LOC': 0, 'B-MISC': 'I-MISC'}
    ner = []
    try_save = ''
    start = time.time()
    i_book = 0
    result = []

    model.to('cuda')

    for book in tqdm(data.values()):
        texts = []
        i_book += 1
        texts.append(book['Description']) 
        for review in book['Review'].values():
            texts.append(review['Content'])
            for comment in review['Comment']:
                texts.append(comment[2])

        # To perform word segmentation, POS tagging, NER and then dependency parsing
        for text in texts:
            if len(text)>0:
                is_vn = is_vietnamese(text, vowels)
                # if is_vn:
                #     text = normalize(text)
                #     # Vietnamese NER by using VnCoreNLP
                #     annotated_text = annotator.annotate(text)   
                #     result = annotated_text['sentences']
                #     get_dict_ner(result, ner_dict)
  
                if not is_vn:
                  sequence = text
                  len_sequence = len(text)
                  divide = 1
                  while len_sequence > 500:
                    divide *= 2
                    len_sequence = int(len_sequence/2)
                  for i in range(divide):
                    sequence = text[i*len_sequence:(i+1)*len_sequence]

                    # Bit of a hack to get the tokens with the special tokens
                    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
                    inputs = tokenizer.encode(sequence, return_tensors="pt").to('cuda')

                    outputs = torch.softmax(model(inputs)[0], dim=2)
                    predictions = torch.argmax(outputs, dim=2).squeeze()

                    words = join_token(predictions, tokens)
                    entities = join_word(words)
                    
                    for entity in entities:
                      add_to_dict(entity[1], ner_dict)

    with open('name_entity.json', 'w', encoding='utf-8') as outfile:
        json.dump(ner_dict, outfile, indent=2, separators=(", ", ": "), ensure_ascii=False)
    
    print(time.time() - start)
    print(ner_dict)
    print(count)


11.44284439086914
{'Nguyễn Nhật Ánh': 929, 'Ánh': 1213, 'Goodreads': 3, 'Bêto': 7, 'Hải': 55, 'thành phố Tuổi Thơ': 1, 'Học Yêu': 1, 'Uầyy': 1, 'Nguyễn Ngọc Thuần': 2, 'Nghĩa': 1, 'Khang': 1, 'Phát': 2, 'Nhựt': 1, 'Châu': 7, 'Ướt': 1, 'Mi Mọi': 1, 'Năm': 1, 'khu phố Mỹ Thạnh An': 1, 'Mùi': 2, 'Tôi Xin Một Vé': 5, 'Quy Nhơn': 6, 'Dượng Tony': 1, 'Hải Cồ': 1, 'Trịnh Công Sơn': 3, 'ASEAN': 2, 'Bằng': 1, 'Bạch Tuyết': 1, 'Lynk Lee': 1, 'Cu Mùi': 2, 'Hải Cò': 10, 'Tí Sún': 8, 'Tủn': 3, 'Trái Đất': 5, 'Tun': 3, 'Reading': 1, 'Lan': 10, 'Nga': 8, 'Tuấn': 1, 'Antoine De Saint Exupéry': 1, 'Antoine De Saint Exupér': 1, 'UNESCO': 1, 'Goodread': 2, 'Việt Nam': 72, 'phương Đông': 1, 'Kim Dung': 55, 'Trâm': 106, 'Quỳnh': 140, 'Nguyễn Nhật Anh': 9, 'thành phố Hồ Chí Minh': 2, 'quán Đo Đo': 9, 'Quảng Nam': 14, 'Trái đất': 1, 'Robert Rojdesvensky': 1, 'Tũn': 1, 'Aiko': 1, 'Cái Tủn': 1, 'Rojdesvensky': 1, 'Tún': 1, 'Maputo': 1, 'Nhật Bản': 1, 'Hà Lan': 700, 'Ngạn': 873, 'Hà lan': 6, 'Mắt Biếc': 17, 'T